In [1]:
# Import the necessary Python libraries
import numpy as np
import matplotlib.pyplot as plt
import json

In [5]:
# Configuration name.
CONFIG_FILE = "load-config.json"
CONFIG = json.load(open(CONFIG_FILE))

# Results directory.
DATA_DIR = "results/"

# Service names parsed from load-config.json
SERVICES = []
for service in CONFIG["services"]:
    SERVICES.append(service["name"])
    
# Service hit rates.
# Number of workers is equal to rate / 10.
RATES = [10, 1000]


In [ ]:
def read_datafile(idx, name, rate):
    """
    Reads the complete CSV file for a single load test, returning 
    an array containing the latency at each timestep.
    """
    
    # Make file name.
    fileName = DATA_DIR + str(idx) + "-" + str(name) + "-rate" + str(rate) + ".csv"
    
    # Names of all headers.
    # headerNames=['time (unix epoch)', 'status', 'latency (ns)', 'bytes in', 'bytes out', 'error']
    headerNames=None
    
    # Custom converter for transforming binary tuple mask into a reversed string representation.
    customConverters = {2: lambda x: str(x).count("1")}
    
    # Read data
    try:
        return np.loadtxt(fileName, delimiter=",", usecols=[1, 3], dtype='int, int', converters=customConverters)
    except FileNotFoundError:
        print(fname)
        print("File not found! Returning empty array.")
        return np.empty()
    
def parseLoadTests(robots, density, topology, walls, move, aprob, wprob, vtsize, vtsactive, vtspassive, source="cluster"):
    """
    Parses ten complete iterations of an experiment, returning an array that
    represents the median, max, and min number of tuples present across all
    robots in the swarm at each point in each experiment.
    """
    
    # Create holder for the final data arrays.
    finalData = []
    
    # Iterate over all ten seeds (1-indexed).
    for i in range(1, 11):
        # Read in a set of data.
        data = read_datafile(robots, density, topology, walls, move, aprob, wprob, vtsize, vtsactive, vtspassive, i, source)

        # Take median of every Robots'th row, smoothing every tenth row.
        rowMedian = np.median(data.reshape(-1, robots), axis=1)
        # rowMedian = np.min(rowMedian.reshape(-1, 20), axis=1) # Smooth wildly different values.
        finalData.append(rowMedian)
        
        # Take max of every Robots'th row, smoothing every tenth row.
        rowMax = np.max(data.reshape(-1, robots), axis=1)
        # rowMax = np.min(rowMax.reshape(-1, 20), axis=1) # Smooth wildly different values.
        finalData.append(rowMax)
        
        # Take min of every Robots'th row, smoothing every tenth row.
        rowMin = np.min(data.reshape(-1, robots), axis=1)
        # rowMin = np.min(rowMin.reshape(-1, 20), axis=1) # Smooth wildly different values.
        finalData.append(rowMin)
    
    # Return all runs.
    return np.array(finalData)

def plotTuplesOverTime(data, label):
    """
    Creates a figure containing plots for 
    each iteration of a single experiment data set.
    """
    
    # Create a new figure for the plots.
    fig = plt.figure(figsize=(50,30))
    fig.suptitle(label)
    
    # Create a plot for each service.
    for service in services:
        
        # Create the plot.
        p = fig.add_subplot(2, 5, i + 1)
        
        # Graph median, max, and min data.
        p.plot(data[i * 3], label="Median")
        p.plot(data[i * 3 + 1], label="Max")
        p.plot(data[i * 3 + 2], label="Min")
        
        # Create legend.
        p.legend(loc="upper left")
        
        # Label the plot.
        p.set_title("Seed: " + str(i))

    # Provide axis labels for the figure.
    plt.ylabel("Tuples")
    plt.xlabel("Steps")
    
    # Show the figure.
    plt.show()

In [ ]:
def createAllStaticFigures():
    """
    Creates plot figures for every static experiment.
    """
    
    # Iterate over all configurations of robots, request probabilities, and topologies.
    for robots in ROBOTS:
        for aprob in APROBS:
            for top in TOPOLOGIES:
                
                # Parse and plot.
                plotTuplesOverTime(parseTuplesOverTimeForExperiment(robots, 1, top, False, False, aprob, WPROBS[0], VS_TABLE_SIZES[0], VS_TABLE_ACTIVES[0], VS_TABLE_PASSIVES[0]), 
                                   "Static " + str(robots) + 
                                   " Robots, " + str(aprob) + 
                                   " Access Rate, " + str(top) + 
                                   " Topology")
                
createAllStaticFigures()